In [ ]:

import re
from tabulate import tabulate

# ==== 1. Вхідні дані для STR  ====
data = """
ГВМ1\t14 [0-16,31] 3 [16,31-21,75] 6 [21,75-26,51] 2 [26,51-37,38] 3 [37,38-42,14] 5 [42,14-53,01] 3 [53,01-55,05] 11 [55,05-65,92] 5 [65,92-70,68] 13 [70,68-76,12] 1 [81,54-86,98] 4 [95,13-100,57] 7 [120,95-133,19] 4 [133,19-148,82] 7 [150,85-161,72] 9 [164,44-169,2] 8 [176,67-187,54] 13 [236,48-247,35] 10 [247,35-258,22] 14 [258,22-260,26] 12 [289,47-300,34]
ГВМ2\t2 [0-4,08] 5 [4,08-16,31] 11 [16,31-28,54] 3 [28,54-31,94] 6 [31,94-44,17] 13 [44,17-56,4] 5 [56,4-59,8] 1 [59,8-73,39] 4 [73,39-86,98] 5 [86,98-99,21] 7 [99,21-112,8] 4 [112,8-125,03] 8 [125,03-138,62] 7 [138,62-150,85] 9 [150,85-164,44] 8 [164,44-176,67] 10 [176,67-190,26] 8 [190,26-194,34] 11 [194,34-207,93] 13 [207,93-221,52] 10 [221,52-233,75] 14 [233,75-247,34] 12 [247,34-273,16] 10 [273,16-277,24] 13 [277,24-281,32] 2 [281,32-310,54] 12 [310,54-314,62]
ГВМ3\t3 [42,14-51,65] 5 [70,68-77,48] 1 [86,98-99,21] 8 [138,62-154,93] 9 [169,2-188,23] 10 [190,26-206,57] 11 [207,93-226,96] 13 [226,96-236,48] 10 [236,48-245,99] 14 [247,34-256,85] 12 [273,16-289,47]
ГВМ4\t3 [0-8,15] 6 [8,15-16,3] 1 [73,39-81,54] 4 [86,98-95,13] 7 [112,8-120,95] 8 [154,93-163,08] 10 [206,57-214,72]
ГВМ5\t6 [16,3-17,66]
"""

# ==== 2. Витягуємо часові межі ====
matches = re.findall(r'\[(\d+(?:,\d+)?)-(\d+(?:,\d+)?)\]', data)
unique_values = set()
for start, end in matches:
    unique_values.add(float(start.replace(',', '.')))
    unique_values.add(float(end.replace(',', '.')))
sorted_values = sorted(unique_values)

# ==== 3. Кількість ГВМ ====
gvm_ids = sorted(set(re.findall(r'ГВМ(\d+)', data)))
num_modules = len(gvm_ids)

# ==== 4. Порожня таблиця ====
table = [['' for _ in range(num_modules)] for _ in sorted_values]

def fill_table(gvm_num, op_id, start, end, idx):
    start_f = float(start.replace(',', '.'))
    end_f = float(end.replace(',', '.'))
    duration = round(end_f - start_f, 2)
    gvm_index = int(gvm_num) - 1

    for i, val in enumerate(sorted_values):
        if start_f == val:
            mark = f"*Д{op_id}_{idx}[{duration}]=[{duration}]"
        elif end_f == val:
            mark = f"-Д{op_id}_{idx}[{duration}]=[{duration}]"
        elif start_f < val < end_f:
            mark = f"+Д{op_id}_{idx}[{duration}]=[{duration}]"
        else:
            continue
        current = table[i][gvm_index]
        formatted_mark = current + ";\n" + mark if current else mark
        table[i][gvm_index] = formatted_mark



# ==== 6. Побудова словника операцій ====
op_entries = {}
for line in data.strip().split('\n'):
    gvm_match = re.match(r'ГВМ(\d+)\t(.+)', line)
    if not gvm_match:
        continue
    gvm_num, ops = gvm_match.groups()
    matches = re.findall(r'\b(\d{1,2})\s+\[(\d+(?:,\d+)?)-(\d+(?:,\d+)?)\]', ops)
    for op, start, end in matches:
        if op not in op_entries:
            op_entries[op] = []
        op_entries[op].append((gvm_num, op, start, end))



# ==== 7. Додаємо очікування і заповнюємо таблицю ====
for op_id in sorted(op_entries, key=int):
    entries = sorted(op_entries[op_id], key=lambda x: float(x[2].replace(',', '.')))
    last_end = 0.0  # час завершення попередньої операції цієї деталі
    for idx, (gvm_num, op, start, end) in enumerate(entries, start=1):
        start_f = float(start.replace(',', '.'))
        end_f = float(end.replace(',', '.'))
        op_duration = round(end_f - start_f, 2)

        # === Додаємо період очікування (без символів) ===
        if start_f > last_end:
            gvm_index = int(gvm_num) - 1
            for i, val in enumerate(sorted_values):
                if last_end <= val <= start_f:
                    mark = f"Д{op_id}_{idx}[{op_duration}]"
                    current = table[i][gvm_index]
                    formatted_mark = current + ";\n" + mark if current else mark
                    table[i][gvm_index] = formatted_mark




        # === Додаємо саму операцію ===
        fill_table(gvm_num, op_id, start, end, idx)
        last_end = end_f


# ==== 7. Виведення операцій і заповнення таблиці ====
for op_id in sorted(op_entries, key=int):
    entries = sorted(op_entries[op_id], key=lambda x: float(x[2].replace(',', '.')))
    print(f"\nОперація '{op_id}' — всі входження з номерами ГВМ:")
    for idx, (gvm_num, op, start, end) in enumerate(entries, start=1):
        start_f = float(start.replace(',', '.'))
        end_f = float(end.replace(',', '.'))
        duration = round(end_f - start_f, 2)
        symbol = '*' if idx == 1 else ('-' if idx == len(entries) else '+')
        print(f"{idx}. ГВМ{gvm_num} {op} [{start}-{end}] ({duration}) → {symbol}")


# ==== 8. Формуємо фінальну таблицю ====
header = ['Час'] + [f"Стан ГВМ{i}" for i in range(1, num_modules + 1)]
final_table = []
for i, t in enumerate(sorted_values):
    row = [f"{t:.2f}"] + table[i]
    final_table.append(row)

# ==== 9. Вивід ====
print("\nТаблиця операцій з урахуванням черги для правила найкоротшої операції:\n")
print(tabulate(final_table, headers=header, tablefmt="grid"))

from docx import Document
from docx.shared import Cm

doc = Document()

# Додаємо таблицю
doc_table = doc.add_table(rows=1, cols=len(header))
doc_table.style = 'Table Grid'
doc_table.autofit = False  # Вимикаємо автопідбір ширини

# Задаємо ширину стовпчиків
column_widths = [Cm(1.0)] + [Cm(4.5)] * (len(header) - 1)

# Заголовки
hdr_cells = doc_table.rows[0].cells
for i, h in enumerate(header):
    hdr_cells[i].text = h
    hdr_cells[i].width = column_widths[i]

# Дані
for row in final_table:
    row_cells = doc_table.add_row().cells
    for i, cell in enumerate(row):
        row_cells[i].text = cell
        row_cells[i].width = column_widths[i]

# Шлях збереження
save_path = r"C:\Users\medal\Desktop\lab1\оп.docx"
doc.save(save_path)
print(f"\n✅ Таблиця найкоротших операцій збережена у файл: {save_path}")

"""


_IncompleteInputError: incomplete input (2411426292.py, line 145)

In [ ]:
import re
from tabulate import tabulate
from docx import Document
from docx.shared import Cm
#
# ==== 1. Вхідні дані для MWKR  ====
data = """
ГВМ1\t14 [0-16,31] 3 [16,31-21,75] 6 [21,75-26,51] 7 [48,92-61,16] 13 [78,82-84,26] 12 [84,26-95,13] 11 [95,13-106] 4 [112,79-118,23] 5 [118,23-129,1] 2 [129,1-139,97] 1 [142,69-148,13] 13 [157,65-168,52] 9 [168,52-173,28] 10 [201,82-212,69] 4 [228,99-244,62] 5 [244,62-249,38] 8 [249,38-260,25] 14 [267,72-269,76] 7 [270,44-281,31] 3 [281,31-286,07] 3 [295,58-297,62]
ГВМ2\t10 [0-13,59] 8 [13,59-27,18] 7 [27,18-40,77] 12 [40,77-66,59] 13 [66,59-78,82] 11 [78,82-91,05] 4 [91,05-104,64] 5 [104,64-116,87] 2 [116,87-120,95] 1 [120,95-134,54] 13 [134,54-148,13] 9 [148,13-161,72] 10 [161,72-173,95] 11 [173,95-187,54] 2 [187,54-216,76] 4 [216,76-228,99] 5 [228,99-232,39] 8 [232,39-244,62] 14 [244,62-258,21] 7 [258,21-270,44] 3 [270,44-273,84] 5 [273,84-286,07] 6 [286,07-298,3] 8 [298,3-302,38] 10 [302,38-306,46] 12 [306,46-310,54] 13 [310,54-314,62]
ГВМ3\t10 [13,59-29,9] 8 [29,9-46,21] 12 [66,59-82,9] 13 [148,13-157,65] 1 [157,65-169,88] 9 [173,28-192,31] 10 [192,31-201,82] 11 [201,82-220,85] 5 [249,38-256,18] 14 [258,21-267,72] 3 [286,07-295,58]
ГВМ4\t3 [0-8,15] 6 [8,15-16,3] 10 [29,9-38,05] 7 [40,77-48,92] 8 [48,92-57,07] 4 [104,64-112,79] 1 [134,54-142,69]
ГВМ5\t6 [16,3-17,66]
"""




# ==== 2. Витягуємо всі часи ====
matches = re.findall(r'\[(\d+(?:,\d+)?)-(\d+(?:,\d+)?)\]', data)
unique_values = {float(x.replace(',', '.')) for pair in matches for x in pair}
sorted_values = sorted(unique_values)

# ==== 3. ГВМ-модулі ====
gvm_ids = sorted(set(re.findall(r'ГВМ(\d+)', data)))
num_modules = len(gvm_ids)

# ==== 4. Порожня таблиця ====
table = [['' for _ in range(num_modules)] for _ in sorted_values]

# ==== 5. Побудова словника операцій ====
op_entries = {}
for line in data.strip().split('\n'):
    gvm_match = re.match(r'ГВМ(\d+)\t(.+)', line)
    if not gvm_match:
        continue
    gvm_num, ops = gvm_match.groups()
    matches = re.findall(r'\b(\d{1,2})\s+\[(\d+(?:,\d+)?)-(\d+(?:,\d+)?)\]', ops)
    for op, start, end in matches:
        if op not in op_entries:
            op_entries[op] = []
        op_entries[op].append((gvm_num, op, start, end))

# ==== 7. Додаємо очікування і заповнюємо таблицю з урахуванням накопиченого часу ==== 
for op_id in sorted(op_entries, key=int):
    entries = sorted(op_entries[op_id], key=lambda x: float(x[2].replace(',', '.')))
    total_op_time = sum(round(float(e[3].replace(',', '.')) - float(e[2].replace(',', '.')), 2) for e in entries)
    remaining_total = round(total_op_time, 2)
    last_end = 0.0

    for idx, (gvm_num, _, start, end) in enumerate(entries, start=1):
        start_f = float(start.replace(',', '.'))
        end_f = float(end.replace(',', '.'))
        duration = round(end_f - start_f, 2)
        gvm_index = int(gvm_num) - 1

        # === Додаємо очікування (проміжок між попереднім кінцем і новим стартом) ===

        if start_f > last_end:
            for i, t in enumerate(sorted_values):
                if last_end <= t < start_f:
                    wait_label = f"Д{op_id}_{idx}[{remaining_total:.2f}]"
                    current = table[i][gvm_index]
                    table[i][gvm_index] = f"{current};\n{wait_label}" if current else wait_label


        # === Додаємо позначки операції ===
        label = f"Д{op_id}_{idx}[{remaining_total:.2f}]=[{duration}]"
        for i, t in enumerate(sorted_values):
            if start_f == t:
                symbol = '*'
            elif end_f == t:
                symbol = '-'
            elif start_f < t < end_f:
                symbol = '+'
            else:
                continue
            mark = f"{symbol}{label}"
            current = table[i][gvm_index]
            table[i][gvm_index] = f"{current};\n{mark}" if current else mark

        remaining_total = round(remaining_total - duration, 2)
        last_end = end_f


# ==== 7. Формування фінальної таблиці ====
header = ['Час'] + [f"Стан ГВМ{i}" for i in range(1, num_modules + 1)]
final_table = [[f"{t:.2f}"] + row for t, row in zip(sorted_values, table)]

print("\n🔧 Таблиця операцій з урахуванням накопичених часів:\n")
print(tabulate(final_table, headers=header, tablefmt="grid"))

# ==== 8. Збереження у Word ====
doc = Document()
doc_table = doc.add_table(rows=1, cols=len(header))
doc_table.style = 'Table Grid'
doc_table.autofit = False

column_widths = [Cm(1.0)] + [Cm(4.5)] * (len(header) - 1)
hdr_cells = doc_table.rows[0].cells
for i, h in enumerate(header):
    hdr_cells[i].text = h
    hdr_cells[i].width = column_widths[i]

for row in final_table:
    row_cells = doc_table.add_row().cells
    for i, cell in enumerate(row):
        row_cells[i].text = cell
        row_cells[i].width = column_widths[i]

save_path = r"C:\Users\medal\Desktop\lab1\оп_2.docx"
doc.save(save_path)
print(f"\n✅ Файл збережено: {save_path}")



🔧 Таблиця операцій з урахуванням накопичених часів:

+--------+------------------------+------------------------+------------------------+----------------------+---------------------+
|    Час | Стан ГВМ1              | Стан ГВМ2              | Стан ГВМ3              | Стан ГВМ4            | Стан ГВМ5           |
+========+========================+========================+========================+======================+=====================+
|   0    | *Д14_1[41.45]=[16.31]  | Д1_1[39.41];           |                        | Д3_1[33.30];         |                     |
|        |                        | Д2_1[44.17];           |                        | *Д6_1[26.50]=[8.15]  |                     |
|        |                        | Д4_1[55.04];           |                        |                      |                     |
|        |                        | Д5_1[50.29];           |                        |                      |                     |
|        |                   

In [ ]:
import re
from tabulate import tabulate
from docx import Document
from docx.shared import Cm
#
# ==== 1. Вхідні дані для LUKR  ====
data = """
ГВМ1\t14 [0-16,31] 6 [16,31-21,07] 9 [21,07-25,83] 3 [25,83-31,27] 1 [35,33-40,77] 3 [42,81-47,57] 2 [60,48-71,35] 3 [71,35-73,39] 5 [73,39-84,26] 14 [84,26-86,3] 5 [105,33-110,09] 4 [127,07-132,51] 11 [143,38-154,25] 4 [155,61-171,24] 13 [181,43-186,87] 12 [223,56-234,43] 13 [234,43-245,3] 7 [245,3-257,54] 7 [282-292,87] 8 [294,23-305,1] 10 [315,97-326,84]
ГВМ2\t9 [0-13,59] 1 [13,59-27,18] 6 [27,18-39,41] 3 [39,41-42,81] 14 [42,81-56,4] 2 [56,4-60,48] 5 [60,48-72,71] 2 [72,71-101,93] 5 [101,93-105,33] 4 [105,33-118,92] 5 [118,92-131,15] 11 [131,15-143,38] 4 [143,38-155,61] 11 [155,61-169,2] 13 [169,2-181,43] 12 [181,43-207,25] 13 [207,25-220,84] 7 [220,84-234,43] 12 [234,43-238,51] 8 [238,51-252,1] 13 [252,1-256,18] 10 [256,18-269,77] 7 [269,77-282] 8 [282-294,23] 10 [294,23-306,46] 8 [306,46-310,54] 10 [326,84-330,92]
ГВМ3\t9 [25,83-44,86] 1 [44,86-57,09] 3 [57,09-66,6] 14 [66,6-76,11] 5 [110,09-116,89] 11 [169,2-188,23] 12 [207,25-223,56] 13 [223,56-233,08] 8 [252,1-268,41] 10 [269,77-286,08] 10 [306,46-315,97]
ГВМ4\t6 [0-8,15] 3 [8,15-16,3] 1 [27,18-35,33] 4 [118,92-127,07] 7 [234,43-242,58] 8 [268,41-276,56] 10 [286,08-294,23]
ГВМ5\t6 [8,15-9,51]
"""




# ==== 2. Витягуємо всі часи ====
matches = re.findall(r'\[(\d+(?:,\d+)?)-(\d+(?:,\d+)?)\]', data)
unique_values = {float(x.replace(',', '.')) for pair in matches for x in pair}
sorted_values = sorted(unique_values)

# ==== 3. ГВМ-модулі ====
gvm_ids = sorted(set(re.findall(r'ГВМ(\d+)', data)))
num_modules = len(gvm_ids)

# ==== 4. Порожня таблиця ====
table = [['' for _ in range(num_modules)] for _ in sorted_values]

# ==== 5. Побудова словника операцій ====
op_entries = {}
for line in data.strip().split('\n'):
    gvm_match = re.match(r'ГВМ(\d+)\t(.+)', line)
    if not gvm_match:
        continue
    gvm_num, ops = gvm_match.groups()
    matches = re.findall(r'\b(\d{1,2})\s+\[(\d+(?:,\d+)?)-(\d+(?:,\d+)?)\]', ops)
    for op, start, end in matches:
        if op not in op_entries:
            op_entries[op] = []
        op_entries[op].append((gvm_num, op, start, end))

# ==== 7. Додаємо очікування і заповнюємо таблицю з урахуванням накопиченого часу ==== 
for op_id in sorted(op_entries, key=int):
    entries = sorted(op_entries[op_id], key=lambda x: float(x[2].replace(',', '.')))
    total_op_time = sum(round(float(e[3].replace(',', '.')) - float(e[2].replace(',', '.')), 2) for e in entries)
    remaining_total = round(total_op_time, 2)
    last_end = 0.0

    for idx, (gvm_num, _, start, end) in enumerate(entries, start=1):
        start_f = float(start.replace(',', '.'))
        end_f = float(end.replace(',', '.'))
        duration = round(end_f - start_f, 2)
        gvm_index = int(gvm_num) - 1

        # === Додаємо очікування (проміжок між попереднім кінцем і новим стартом) ===

        if start_f > last_end:
            for i, t in enumerate(sorted_values):
                if last_end <= t < start_f:
                    wait_label = f"Д{op_id}_{idx}[{remaining_total:.2f}]"
                    current = table[i][gvm_index]
                    table[i][gvm_index] = f"{current};\n{wait_label}" if current else wait_label


        # === Додаємо позначки операції ===
        label = f"Д{op_id}_{idx}[{remaining_total:.2f}]=[{duration}]"
        for i, t in enumerate(sorted_values):
            if start_f == t:
                symbol = '*'
            elif end_f == t:
                symbol = '-'
            elif start_f < t < end_f:
                symbol = '+'
            else:
                continue
            mark = f"{symbol}{label}"
            current = table[i][gvm_index]
            table[i][gvm_index] = f"{current};\n{mark}" if current else mark

        remaining_total = round(remaining_total - duration, 2)
        last_end = end_f


# ==== 7. Формування фінальної таблиці ====
header = ['Час'] + [f"Стан ГВМ{i}" for i in range(1, num_modules + 1)]
final_table = [[f"{t:.2f}"] + row for t, row in zip(sorted_values, table)]

print("\n🔧 Таблиця операцій з урахуванням накопичених часів:\n")
print(tabulate(final_table, headers=header, tablefmt="grid"))

# ==== 8. Збереження у Word ====
doc = Document()
doc_table = doc.add_table(rows=1, cols=len(header))
doc_table.style = 'Table Grid'
doc_table.autofit = False

column_widths = [Cm(1.0)] + [Cm(4.5)] * (len(header) - 1)
hdr_cells = doc_table.rows[0].cells
for i, h in enumerate(header):
    hdr_cells[i].text = h
    hdr_cells[i].width = column_widths[i]

for row in final_table:
    row_cells = doc_table.add_row().cells
    for i, cell in enumerate(row):
        row_cells[i].text = cell
        row_cells[i].width = column_widths[i]

save_path = r"C:\Users\medal\Desktop\lab1\оп_3.docx"
doc.save(save_path)
print(f"\n✅ Файл збережено: {save_path}")


In [63]:
import re
from docx import Document
from docx.shared import Inches

# ==== 1. Вхідні дані ====
data = """
ГВМ1\t14 [0-16,31] 3 [16,31-21,75] 6 [21,75-26,51] 7 [48,92-61,16] 13 [78,82-84,26] 12 [84,26-95,13] 11 [95,13-106] 4 [112,79-118,23] 5 [118,23-129,1] 2 [129,1-139,97] 1 [142,69-148,13] 13 [157,65-168,52] 9 [168,52-173,28] 10 [201,82-212,69] 4 [228,99-244,62] 5 [244,62-249,38] 8 [249,38-260,25] 14 [267,72-269,76] 7 [270,44-281,31] 3 [281,31-286,07] 3 [295,58-297,62]
ГВМ2\t10 [0-13,59] 8 [13,59-27,18] 7 [27,18-40,77] 12 [40,77-66,59] 13 [66,59-78,82] 11 [78,82-91,05] 4 [91,05-104,64] 5 [104,64-116,87] 2 [116,87-120,95] 1 [120,95-134,54] 13 [134,54-148,13] 9 [148,13-161,72] 10 [161,72-173,95] 11 [173,95-187,54] 2 [187,54-216,76] 4 [216,76-228,99] 5 [228,99-232,39] 8 [232,39-244,62] 14 [244,62-258,21] 7 [258,21-270,44] 3 [270,44-273,84] 5 [273,84-286,07] 6 [286,07-298,3] 8 [298,3-302,38] 10 [302,38-306,46] 12 [306,46-310,54] 13 [310,54-314,62]
ГВМ3\t10 [13,59-29,9] 8 [29,9-46,21] 12 [66,59-82,9] 13 [148,13-157,65] 1 [157,65-169,88] 9 [173,28-192,31] 10 [192,31-201,82] 11 [201,82-220,85] 5 [249,38-256,18] 14 [258,21-267,72] 3 [286,07-295,58]
ГВМ4\t3 [0-8,15] 6 [8,15-16,3] 10 [29,9-38,05] 7 [40,77-48,92] 8 [48,92-57,07] 4 [104,64-112,79] 1 [134,54-142,69]
ГВМ5\t6 [16,3-17,66]
"""

# ==== 2. Створення документа Word ====
doc = Document()
doc.add_heading("Локальні резерви між операціями по кожному ГВМ", 0)

# ==== 3. Парсинг та запис таблиць ====
gvm_blocks = re.findall(r'ГВМ(\d+)\t([^\n]+)', data)

for gvm_num, ops_str in gvm_blocks:
    ops = re.findall(r'(\d+)\s+\[(\d+(?:,\d+)?)-(\d+(?:,\d+)?)\]', ops_str)
    ops = [(op, float(start.replace(',', '.')), float(end.replace(',', '.'))) for op, start, end in ops]
    ops.sort(key=lambda x: x[1])

    reserves = []

    # Перевірка першої паузи
    first_op = ops[0]
    if first_op[1] > 0:
        reserves.append(["ГВМ" + gvm_num, f"{first_op[0]} (перед початком)", "0.00", f"{first_op[1]:.2f}", f"{first_op[1]:.2f}"])

    # Перевірка міжопераційних резервів
    for i in range(1, len(ops)):
        prev_end = ops[i - 1][2]
        curr_start = ops[i][1]
        if curr_start > prev_end:
            reserves.append([
                "ГВМ" + gvm_num,
                f"{ops[i][0]} (після {ops[i - 1][0]})",
                f"{prev_end:.2f}",
                f"{curr_start:.2f}",
                f"{curr_start - prev_end:.2f}"
            ])

    # Додаємо таблицю до документа
    doc.add_heading(f" Резерви для ГВМ{gvm_num}", level=2)

    if reserves:
        table = doc.add_table(rows=1, cols=5)
        table.style = "Table Grid"
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = "ГВМ"
        hdr_cells[1].text = "Операція"
        hdr_cells[2].text = "Очікування з"
        hdr_cells[3].text = "до"
        hdr_cells[4].text = "Тривалість"

        for row in reserves:
            row_cells = table.add_row().cells
            for i in range(5):
                row_cells[i].text = row[i]
    else:
        doc.add_paragraph("Немає резервів для цього ГВМ.")

# ==== 4. Збереження ====
output_path = r"C:\Users\medal\Desktop\lab1\резерви_по_гвм.docx"
doc.save(output_path)
print(f"✅ Документ збережено: {output_path}")


✅ Документ збережено: C:\Users\medal\Desktop\lab1\резерви_по_гвм.docx
